In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import seaborn as sns

https://drive.google.com/drive/folders/1J9ttwxBkPqeNuzaV6fipV5OwjqegXRwJ

https://docs.google.com/spreadsheets/d/12bfWcjWKDXxDLplgPzqWFU3U89CPn7Vd8ugst9hMfD4/edit#gid=0

**Гипотезы 3, 4**

Одна нормальная выборка, дисперсия неизвестна. Односторонняя и двусторонняя альтернативы.

Одна нормальная выборка, дисперсия неизвестна. Двусторонняя альтернатива.

$$ X_1, .. X_n \propto \mathcal{N}(\theta_1, \theta_2^{-1}), \theta = (\theta_1, theta_2) $$.
$$H_0: \theta_1 = t_0, H_1: \theta_1 \neq t_0$$

$q_2(t_2) \propto 1 / t_2, \theta \sim \mathcal{N}(a, \sigma_{pr}^2 )$ при $H_1$, $\sigma_{pr}^2 = k \theta_2 ^ {-1}, k$ известно. Если ввести дополнительно 
переменные $v = n - 1$, $t = \frac{t_1 - \overline{x}}{s}\sqrt{n-1}$
Тогда можно вывести 
$$p_{t_1^0}(x) = \int\limits_0^\infty p_{t_1^0, t_2} (x) q_2(t_2) dt_2 \propto
(1 + t^2/v)^{-\frac{v+1}{2}}
$$
$$p_1(x) = \int\limits_0^{\infty} p_{t_1^0, t_2} (x) q_1(t_1) q_2(t_2) dt_1 dt_2 \propto
\frac{1}{\sqrt{1 + nk}}(1 + \frac{1}{v}\frac{t^2}{1 + nk})^{-\frac{v+1}{2}}
$$

Будет интересно рассмотреть метод Линли. Для этого можно расписать совместную плотность:
$p(t_1, t_2 | X) = t_2^{n/2 - 1}e^{-\frac{t_2}{2}\sum(x_i - t_1)^2}$.
Ввиду того, что требуется оценивать распределение $t_1$, делаем сначала интегрирование по $t_2 \in [0, +\inf)$ 
$$ \int\limits_0^\infty t_2^{n/2 - 1}e^{-\frac{t_2}{2}\sum(x_i - t_1)^2} d t_2 = 
[u = \sum(x_i - t_1)^2, t = \frac{t_2}{2}u, t_2 = \frac{2t}{u},
d t_2 = \frac{2}{u} dt
]
= \int\limits_0^\infty \left(\frac{2}{u}\right)^{n/2} t^{n/2 - 1} e^{-t}dt = 
\Gamma(n/2) 2^{n/2} \frac{1}{\left(\sum(x_i - t_1)^2\right)^{n/2}} =
 \frac{\Gamma(n/2) 2^{n/2}}{n^{n/2}\left((\overline{X} - t_1)^2 + S^2\right)^{n/2}} \propto \frac{1}{\left((\overline{X} - t_1)^2 + S^2\right)^{n/2}}
$$
Отсюда можем получить, что 
$P(t_1 \in (\overline{X} - \delta, \overline{X} + \delta)) \propto 
\int\limits_{-\delta}^{\delta}\frac{1}{t_1^2 + S^2}dt_1$, и, очевидно, HDR для простой гипотезы имеет именно вид $(\overline{X} - \delta, \overline{X} + \delta)$. Подобные же интегралы используются и для сложных гипотез.


Более того, исходя того, что равномерный приор от $t_1$ пропорционален константе, мы можем считать, что $\forall t_1^0 \pi_0 = \pi_1 = 0.5$, и Байесовский фактор равен просто отношению апостериорных "вероятностей" гипотез $H_0$ и $H_1$

In [0]:
class Hypotheses1UncnownSigmaUniformPrior:
# Нужно объединить с другим классом, для случая 
  def __init__(self):
    pass

  def test(self, sample, test_params):
    """
    :param sample: - выборка из нормального распределения с неизвестной
     дисперсией
    :param  test_params - словарь с параметрами гипотезы
    обязательно содержит:
    "type_of_test" - одно из "atom", "modification", "lindley", "atom",
    "leq", "geq" - тип гипотезы 
    "t0": собственно, точка, относительно которой проверяется гипотеза
    для метода с атомом: pi_0: - априорная вероятность, k: относится к дисперсии 
    априорного распределения, см. выкладки
    для метода модификации гипотезы: "epsilon" - радиус "окрестности" t0
    для метода Линдли - "conf" - уроветь зачимости
    возвращает метод также словарь, в зависимости от гипотезы может 
    содержать "HDR" в виде концов отрезка, "rejected" - для метода Линдли,
    "p0", "p1", "B" - апостериорные вероятности и байесовский фактор,
    "post_params" - ппараметры постериорного распределения, см. выкладки и примеры
    применения
    """
    t0 = test_params["t0"]
    n = len(sample)

    test_type = test_params["type_of_test"]

    assert test_type in ["modification", "lindley", "atom",
                                  "leq", "geq"], "unknown test type"

    

    if test_type == "atom":
      try:
        self.pi_0 = test_params["pi_0"]
        self.pi_1 = 1 - self.pi_0
        self.k = test_params["k"]
      except Exception:
        print("not enough params")


      v = n - 1
      t = (t0 - np.mean(sample)) * np.sqrt(n - 1) / (np.std(sample) ** 2) 

      self.p0 = (1 + t ** 2 / v) ** (-(v + 1) / 2) * self.pi_0 # почему не - n  / 2 ?
      self.p1 = 1 / np.sqrt(1 + n * self.k) * (1 + t ** 2 / (1 + n * self.k) / v) ** (-(v + 1) / 2) * self.pi_1
      normalizator = self.p0 + self.p1
      self.p0 /= normalizator
      self.p1 /= normalizator
      self.B = self.p0 / self.p1

      self.t0 = t0
      self.n = n
      return {
          "p0":self.p0,
          "p1":self.p1,
          "B": self.B,

      }

    m = np.mean(sample)
    S_sq = np.mean((sample - m) ** 2) 
    poster_distr = lambda x: 1 / ((m - x)**2 + S_sq) ** (n/2)
    
    big_num = 10**4
    step = 1000
    big_space = np.linspace(m-big_num, m+big_num, 2 * big_num * step)
    divisor = np.sum(poster_distr(big_space)) / step
    poster_distr = lambda x: 1 / ((m - x)**2 + S_sq) ** (n/2) / divisor
    pdf_with_step = poster_distr(big_space) / step
    cdf = np.cumsum(pdf_with_step)
    if test_type == "lindley":
        try:
          conf = 1 - test_params["conf"]
        except Exception:
          print("not enough params")
        l, r = big_space[np.argmin((cdf - (1 - conf)/2)**2)], big_space[np.argmin((cdf - (1 + conf)/2)**2)]
        reject = False
        if t0 < l or t0 > r:
          reject = True
        return {
            "reject":reject,
            "HDR": (l, r),
            "post_params": {
                "n": n, 
                "mean": m,
                "S_sq": S_sq
            } 
        }

    if test_type == "modification":
          try:
            epsilon = test_params["epsilon"]
          except Exception:
            print("not enough params")
          p0 = cdf[np.argmin((t0 + epsilon)**2)] - cdf[np.argmin((t0 - epsilon)**2)]

          # под HDR в данном случае подразумеваем такую симметричную окрестность  
          # среднего, что t0 в неё попадает "а границе"

          #Возможно, есть смысл сделать по другому?
          delta = np.abs(m - t0)
          l, r = (m- delta, m + delta)
          return {
              "p0": p0,
              "p1": 1 - p0,
              "HDR": (l, r),
              "post_params": {
                  "n": n, 
                  "mean": m,
                  "S_sq": S_sq
              } 
          }
    
    if test_type in ["leq", "geq"]:
      p0 = cdf[np.argmin((t0 - big_space)**2)]
      p1 = 1 - p0
      if test_type == "geq":
        p0, p1 = p1, p0
      B = p0 / p1
      # При чем тут HDR, не очень понял
      return {
          "p0": p0,
          "p1": p1, 
          "B": B
      }

##Примеры применения.
Один класс может тестировать различные типы гипотез, в том числе и "сложная против сложной"

In [0]:
hypo = Hypotheses1UncnownSigmaUniformPrior()

In [0]:
mean_real = 0
std_real = 1
n = 1000
sample = sps.norm(mean_real, std_real).rvs(n)
t0 = 0.001

In [50]:
test_type_1 = {
    "type_of_test": "atom",
    "t0": 0,
    "pi_0": 0.1,
    "k": 1,
}
hypo.test(sample, test_type_1)

{'B': 3.5077733542064573, 'p0': 0.778160985164251, 'p1': 0.22183901483574892}

In [51]:
test_type_2 = {
    "type_of_test": "modification",
    "t0": 0,
    "epsilon": 0.5,
}
hypo.test(sample, test_type_2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'HDR': (-0.0043942463462630955, 0.0),
 'p0': 0.0,
 'p1': 1.0,
 'post_params': {'S_sq': 1.0537974971047268,
  'mean': -0.0021971231731315478,
  'n': 1000}}

In [52]:
test_type_3 = {
    "type_of_test": "lindley",
    "t0": 0,
    "conf": 0.05
}
hypo.test(sample, test_type_3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'HDR': (-0.06669712639813952, 0.061302880001676385),
 'post_params': {'S_sq': 1.0537974971047268,
  'mean': -0.0021971231731315478,
  'n': 1000},
 'reject': False}

In [53]:
test_type_4 = {
    "type_of_test": "leq",
    "t0": 0
}
hypo.test(sample, test_type_4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'B': 1.1588465886427948, 'p0': 0.5367896888733203, 'p1': 0.46321031112667965}

In [54]:
test_type_5 = {
    "type_of_test": "geq", 
    "t0": 0
}
hypo.test(sample, test_type_5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'B': 0.8629269912000768, 'p0': 0.46321031112667965, 'p1': 0.5367896888733203}

##Тестируем библиотеку

Сложная гипотеза против сложной,  H_0 верна

In [56]:
mean_real = -0.05
std_real = 1

for n in [10, 100, 1000]:
  print("n: ", n)
  sample = sps.norm(mean_real, std_real).rvs(n)
  t0 = 0
  print(hypo.test(sample, test_type_4))

n:  10
{'p0': 0.5197491180324902, 'p1': 0.48025088196750976, 'B': 1.0822450047424435}
n:  100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'p0': 0.43899861637442805, 'p1': 0.561001383625572, 'B': 0.7825267979506946}
n:  1000
{'p0': 0.7502981621896091, 'p1': 0.2497018378103909, 'B': 3.0047762914718392}


Видим рост байесовского фактора. То же, только верно H_1:

In [58]:
mean_real = 0.05
std_real = 1

for n in [10, 100, 1000]:
  print("n: ", n)
  sample = sps.norm(mean_real, std_real).rvs(n)
  t0 = 0
  print(hypo.test(sample, test_type_4))

n:  10
{'p0': 0.2980242555561742, 'p1': 0.7019757444438258, 'B': 0.4245506456812098}
n:  100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'p0': 0.2463582629839594, 'p1': 0.7536417370160406, 'B': 0.32689041872785224}
n:  1000
{'p0': 0.00834416964805232, 'p1': 0.9916558303519477, 'B': 0.008414380667827969}


Можно еще посмотреть, скажем, на метод с атомом:

In [61]:
mean_real = -0.5
std_real = 1

for n in [10, 100, 1000]:
  print("n: ", n)
  sample = sps.norm(mean_real, std_real).rvs(n)
  t0 = 0
  print(hypo.test(sample, test_type_1))

n:  10
{'p0': 0.000820581824054234, 'p1': 0.9991794181759457, 'B': 0.0008212557315804694}
n:  100
{'p0': 9.604507346772854e-07, 'p1': 0.9999990395492654, 'B': 9.604516571437851e-07}
n:  1000
{'p0': 1.9296133714235655e-57, 'p1': 1.0, 'B': 1.9296133714235655e-57}


In [62]:
mean_real = 0.
std_real = 1

for n in [10, 100, 1000]:
  print("n: ", n)
  sample = sps.norm(mean_real, std_real).rvs(n)
  t0 = 0
  print(hypo.test(sample, test_type_1))

n:  10
{'p0': 0.18165155336536692, 'p1': 0.8183484466346331, 'B': 0.22197335879647442}
n:  100
{'p0': 0.5130679182795874, 'p1': 0.48693208172041264, 'B': 1.0536745011066686}
n:  1000
{'p0': 0.7496647041438538, 'p1': 0.25033529585614633, 'B': 2.9946424517565595}


Работает корректно, смотрим на метод Линдли:

In [63]:
mean_real = -0.5
std_real = 1

for n in [10, 100, 1000]:
  print("n: ", n)
  sample = sps.norm(mean_real, std_real).rvs(n)
  t0 = 0
  print(hypo.test(sample, test_type_3))

n:  10
{'reject': False, 'HDR': (-1.9706402765732491, 0.4513598445264506), 'post_params': {'n': 10, 'mean': -0.7591402159977603, 'S_sq': 2.5782105612080892}}
n:  100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'reject': True, 'HDR': (-0.7450553240305453, -0.3630553049297305), 'post_params': {'n': 100, 'mean': -0.553555314453753, 'S_sq': 0.9196614798021693}}
n:  1000
{'reject': True, 'HDR': (-0.5082471373752924, -0.3842471311763802), 'post_params': {'n': 1000, 'mean': -0.4457471342497093, 'S_sq': 1.0071796552288808}}


In [64]:
mean_real = 0
std_real = 1

for n in [10, 100, 1000]:
  print("n: ", n)
  sample = sps.norm(mean_real, std_real).rvs(n)
  t0 = 0
  print(hypo.test(sample, test_type_3))

n:  10
{'reject': False, 'HDR': (-0.964078895676721, 0.4379211744235363), 'post_params': {'n': 10, 'mean': -0.2625788606002151, 'S_sq': 0.8651859203731108}}
n:  100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: RuntimeWarning: overflow encountered in power


{'reject': False, 'HDR': (-0.2589428949067951, 0.17905712699212017), 'post_params': {'n': 100, 'mean': -0.03944288393164883, 'S_sq': 1.2029054302180953}}
n:  1000
{'reject': False, 'HDR': (-0.07149901247612434, 0.0465009934232512), 'post_params': {'n': 1000, 'mean': -0.011999009500150731, 'S_sq': 0.9094667069305039}}


Наблюдаем за сужением области HDR.